In [6]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity_project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-136112
Azure region: southcentralus
Subscription id: a0a76bad-11a1-4a2d-9887-97a29122c8ed
Resource group: aml-quickstarts-136112


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    cpu_cluster = ComputeTarget(workspace=ws, name="project-cluster")
    print('Found existing cluster, use it.')
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws,"project-cluster", compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
%%writefile conda_dependencies.yml
 dependencies:
 - python=3.6.2
 - scikit-learn
 - pip:
    - azureml-defaults

Overwriting conda_dependencies.yml


In [9]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

env = Environment.from_conda_specification(name="sklearn-env", file_path="./conda_dependencies.yml")

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.0, 1.0),
        "--max_iter": choice(25, 50, 75, 100, 125, 150, 175)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = ScriptRunConfig(source_directory = ".", compute_target = cpu_cluster, script = 'train.py', environment = env)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config = est, 
                             hyperparameter_sampling = ps,
                             policy = policy,
                             primary_metric_name = 'Accuracy', 
                             primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs = 20,
                             max_concurrent_runs = 4)

In [10]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()

run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: b1fd71de-9676-4195-a4f1-b2ecf3eb024a
Web View: https://ml.azure.com/experiments/udacity_project/runs/b1fd71de-9676-4195-a4f1-b2ecf3eb024a?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-136112/workspaces/quick-starts-ws-136112

Execution Summary
RunId: b1fd71de-9676-4195-a4f1-b2ecf3eb024a
Web View: https://ml.azure.com/experiments/udacity_project/runs/b1fd71de-9676-4195-a4f1-b2ecf3eb024a?wsid=/subscriptions/a0a76bad-11a1-4a2d-9887-97a29122c8ed/resourcegroups/aml-quickstarts-136112/workspaces/quick-starts-ws-136112



{'runId': 'b1fd71de-9676-4195-a4f1-b2ecf3eb024a',
 'target': 'local',
 'status': 'Canceled',
 'startTimeUtc': '2021-01-26T11:42:28.40688Z',
 'endTimeUtc': '2021-01-26T11:58:35.521Z',
 'properties': {'ContentSnapshotId': '55c57458-54aa-4d70-8ba9-eb4b2b9ae72f'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {},
 'submittedBy': 'ODL_User 136112'}

In [11]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
print("Accuracy: {}".format(best_run_metrics['Accuracy']))

Accuracy: 0.9139231193672981


In [12]:
best_run.register_model("project1-hyperdrive-best-model","outputs/project1-hyperdrive-model.joblib")

Model(workspace=Workspace.create(name='quick-starts-ws-136112', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-136112'), name=project1-hyperdrive-best-model, id=project1-hyperdrive-best-model:1, version=1, tags={}, properties={})

In [13]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

raw_df = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [14]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(raw_df)

training_data = x.join(y)
training_data.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
0,57,1,0,0,1,5,1,371,1,999,...,0,0,0,0,1,0,0,0,0,0
1,55,1,0,1,0,5,4,285,2,999,...,1,0,0,0,0,0,0,0,1,0
2,33,1,0,0,0,5,5,52,1,999,...,0,0,0,1,0,0,0,0,0,0
3,36,1,0,0,0,6,5,355,4,999,...,1,0,0,0,1,0,0,0,0,0
4,27,1,0,1,0,7,5,189,2,999,...,0,0,0,0,1,0,0,0,0,0


In [15]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=training_data,
    label_column_name="y",
    n_cross_validations=5)

In [16]:
# Submit your automl run

automl_exp = Experiment(workspace=ws, name="udacity_project_automl")
automl_run = automl_exp.submit(automl_config, show_output=True)

No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_5d480745-916d-45a8-acc0-360c932659ca

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/Auto

In [29]:
# Retrieve and save your best automl model.

best_automl_run, best_automl_model = automl_run.get_output()

print(best_automl_run)

Run(Experiment: udacity_project_automl,
Id: AutoML_5d480745-916d-45a8-acc0-360c932659ca_36,
Type: None,
Status: Completed)


In [28]:
best_automl_model.

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                reg_alpha=0.5789473684210527,
                                                                                                reg_lambda=0.42105263157894735,
                                                                                                silent=True,
                             

In [19]:
best_automl_run.get_metrics()['accuracy']

0.9166919575113809

In [20]:
best_automl_run.register_model(model_name="project1-automl-best-model", model_path="./outputs")

Model(workspace=Workspace.create(name='quick-starts-ws-136112', subscription_id='a0a76bad-11a1-4a2d-9887-97a29122c8ed', resource_group='aml-quickstarts-136112'), name=project1-automl-best-model, id=project1-automl-best-model:1, version=1, tags={}, properties={})

In [21]:
cpu_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

